In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
file_path = "thermal_power-ics_combined_dataset.csv"

df = pd.read_csv(file_path)

df.head()


,timestamp,water_flow_rate,water_pressure,water_temperature,steam_pressure,steam_temperature,fuel_rate_proxy,turbine_rpm,power_output,generator_temperature_proxy
0,2025-01-01 00:00:00,0.009225,4.214906,25.786955,-138.370475,351.260017,2.187727,115.959982,2.495829,1.525131
1,2025-01-01 01:00:00,0.013909,3.159849,25.787047,-121.803943,348.539651,1.878631,95.652031,2.950280,1.460782
2,2025-01-01 02:00:00,0.017954,3.204893,25.778057,-120.212268,347.414834,1.992470,67.057205,1.193699,1.340423
3,2025-01-01 19:00:00,0.015913,2.932667,25.788710,-144.431491,349.680486,1.915851,44.991552,0.247391,1.280755
4,2025-01-01 20:00:00,0.012943,4.967830,25.780906,-178.822406,352.879797,2.003181,51.412502,0.563893,1.569184


In [3]:
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2889 entries, 0 to 2888
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   timestamp                    2889 non-null   object 
 1   water_flow_rate              2889 non-null   float64
 2   water_pressure               2889 non-null   float64
 3   water_temperature            2889 non-null   float64
 4   steam_pressure               2889 non-null   float64
 5   steam_temperature            2889 non-null   float64
 6   fuel_rate_proxy              2889 non-null   float64
 7   turbine_rpm                  2889 non-null   float64
 8   power_output                 2889 non-null   float64
 9   generator_temperature_proxy  2889 non-null   float64
dtypes: float64(9), object(1)
memory usage: 225.8+ KB


,water_flow_rate,water_pressure,water_temperature,steam_pressure,steam_temperature,fuel_rate_proxy,turbine_rpm,power_output,generator_temperature_proxy
count,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000
mean,0.820490,12.946844,25.792206,-139.566584,351.812819,1.939249,71.091328,1.001158,1.456923
std,1.052486,21.853969,0.010914,22.666734,4.611068,0.235748,21.165773,0.922558,0.241032
min,-0.001324,0.961112,25.778033,-179.308933,344.007736,1.308387,36.729906,0.118480,1.066538
25%,0.011221,3.275974,25.786962,-144.352578,348.487837,1.878510,62.129220,0.255507,1.279161
50%,0.015856,4.096680,25.788654,-135.152476,351.947505,2.002425,67.082805,0.650294,1.460453
75%,1.738763,5.153675,25.797104,-125.170418,353.450355,2.100755,89.243608,1.198133,1.585338
max,2.511505,77.731216,25.813221,-95.400057,361.120739,2.190921,116.176417,2.965953,1.878964


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, RobustScaler
import warnings
warnings.filterwarnings('ignore')

In [6]:
print("="*80)
print("LOADING DATA")
print("="*80)
df = pd.read_csv('thermal_power-ics_combined_dataset.csv')
print(f"\nDataset Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
print(df.head())

LOADING DATA

Dataset Shape: (2889, 10)
Columns: ['timestamp', 'water_flow_rate', 'water_pressure', 'water_temperature', 'steam_pressure', 'steam_temperature', 'fuel_rate_proxy', 'turbine_rpm', 'power_output', 'generator_temperature_proxy']

First few rows:
             timestamp  water_flow_rate  water_pressure  water_temperature  \
0  2025-01-01 00:00:00         0.009225        4.214906          25.786955   
1  2025-01-01 01:00:00         0.013909        3.159849          25.787047   
2  2025-01-01 02:00:00         0.017954        3.204893          25.778057   
3  2025-01-01 19:00:00         0.015913        2.932667          25.788710   
4  2025-01-01 20:00:00         0.012943        4.967830          25.780906   

   steam_pressure  steam_temperature  fuel_rate_proxy  turbine_rpm  \
0     -138.370475         351.260017         2.187727   115.959982   
1     -121.803943         348.539651         1.878631    95.652031   
2     -120.212268         347.414834         1.992470    67.057

In [7]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)

In [8]:
print("\n" + "="*80)
print("DATA SUMMARY")
print("="*80)
print(df.describe())

# Check for missing values
print("\n" + "="*80)
print("MISSING VALUES CHECK")
print("="*80)
missing = df.isnull().sum()
if missing.sum() > 0:
    print(missing[missing > 0])
else:
    print("✓ No missing values found")


DATA SUMMARY
                           timestamp  water_flow_rate  water_pressure  \
count                           2889      2889.000000     2889.000000   
mean   2025-07-28 06:27:32.336448768         0.820490       12.946844   
min              2025-01-01 00:00:00        -0.001324        0.961112   
25%              2025-04-15 02:00:00         0.011221        3.275974   
50%              2025-07-28 04:00:00         0.015856        4.096680   
75%              2025-11-09 06:00:00         1.738763        5.153675   
max              2026-02-21 08:00:00         2.511505       77.731216   
std                              NaN         1.052486       21.853969   

       water_temperature  steam_pressure  steam_temperature  fuel_rate_proxy  \
count        2889.000000     2889.000000        2889.000000      2889.000000   
mean           25.792206     -139.566584         351.812819         1.939249   
min            25.778033     -179.308933         344.007736         1.308387   
25%     

In [9]:
duplicates = df.duplicated(subset=['timestamp']).sum()
print(f"\nDuplicate timestamps: {duplicates}")
if duplicates > 0:
    print("Removing duplicates...")
    df = df.drop_duplicates(subset=['timestamp'], keep='first')



Duplicate timestamps: 0


In [10]:
print("\n" + "="*80)
print("TEMPORAL FEATURE ENGINEERING")
print("="*80)

# Extract temporal features
df['hour'] = df['timestamp'].dt.hour
df['day'] = df['timestamp'].dt.day
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['month'] = df['timestamp'].dt.month
df['year'] = df['timestamp'].dt.year
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)

# Time-based cyclical encoding (important for periodic patterns)
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

# Time since start (for trend analysis)
df['time_elapsed_hours'] = (df['timestamp'] - df['timestamp'].min()).dt.total_seconds() / 3600

# Shift categories (useful for power plants)
def get_shift(hour):
    if 6 <= hour < 14:
        return 'morning_shift'
    elif 14 <= hour < 22:
        return 'evening_shift'
    else:
        return 'night_shift'

df['shift'] = df['hour'].apply(get_shift)

print("✓ Temporal features created:")
print(f"  - Hour, Day, Day of Week, Month, Year")
print(f"  - Cyclical encodings (sin/cos)")
print(f"  - Time elapsed, Shift categories")



TEMPORAL FEATURE ENGINEERING
✓ Temporal features created:
  - Hour, Day, Day of Week, Month, Year
  - Cyclical encodings (sin/cos)
  - Time elapsed, Shift categories


In [11]:
print("\n" + "="*80)
print("PHYSICS-BASED FEATURE ENGINEERING")
print("="*80)

# Prevent division by zero
epsilon = 1e-6

# 1. Thermal Efficiency Indicators
df['thermal_efficiency_proxy'] = df['power_output'] / (df['fuel_rate_proxy'] + epsilon)
df['steam_quality'] = df['steam_temperature'] / (df['steam_pressure'].abs() + epsilon)

# 2. Water-Steam Relationship
df['water_steam_temp_diff'] = df['steam_temperature'] - df['water_temperature']
df['water_energy_input'] = df['water_flow_rate'] * df['water_temperature']

# 3. Turbine Performance
df['turbine_load_factor'] = df['power_output'] / (df['turbine_rpm'] + epsilon)
df['specific_power'] = df['power_output'] / (df['steam_pressure'].abs() + epsilon)

# 4. Pressure Relationships
df['pressure_ratio'] = df['water_pressure'] / (df['steam_pressure'].abs() + epsilon)
df['pressure_drop'] = df['water_pressure'] - df['steam_pressure']

# 5. Generator Health Indicators
df['generator_load'] = df['power_output'] / (df['generator_temperature_proxy'] + epsilon)
df['thermal_stress'] = df['generator_temperature_proxy'] * df['power_output']

# 6. System Balance Indicators
df['energy_balance'] = (df['fuel_rate_proxy'] * 100) - df['power_output']
df['mass_flow_balance'] = df['water_flow_rate'] - (df['steam_pressure'] / 100)

# 7. Temperature Gradients
df['temp_gradient_water_steam'] = (df['steam_temperature'] - df['water_temperature']) / (df['water_temperature'] + epsilon)

# 8. Power Efficiency
df['rpm_power_ratio'] = df['turbine_rpm'] / (df['power_output'] + epsilon)
df['fuel_efficiency'] = df['power_output'] / (df['fuel_rate_proxy'] + epsilon)

print("✓ Physics-based features created:")
physics_features = ['thermal_efficiency_proxy', 'steam_quality', 'water_steam_temp_diff',
                   'turbine_load_factor', 'pressure_ratio', 'generator_load', 
                   'energy_balance', 'fuel_efficiency']
print(f"  Total: {len(physics_features)} features")


PHYSICS-BASED FEATURE ENGINEERING
✓ Physics-based features created:
  Total: 8 features


In [12]:
print("\n" + "="*80)
print("STATISTICAL FEATURE ENGINEERING")
print("="*80)

# Define sensor columns for rolling statistics
sensor_cols = ['water_flow_rate', 'water_pressure', 'water_temperature',
               'steam_pressure', 'steam_temperature', 'fuel_rate_proxy',
               'turbine_rpm', 'power_output', 'generator_temperature_proxy']

# Rolling window features
window_sizes = [3, 6, 12]  # hours

print("Creating rolling window features...")
for col in sensor_cols:
    for window in window_sizes:
        # Rolling mean
        df[f'{col}_rolling_mean_{window}h'] = df[col].rolling(window=window, min_periods=1).mean()
        
        # Rolling std (volatility)
        df[f'{col}_rolling_std_{window}h'] = df[col].rolling(window=window, min_periods=1).std()
        
        # Rolling min/max (range detection)
        df[f'{col}_rolling_min_{window}h'] = df[col].rolling(window=window, min_periods=1).min()
        df[f'{col}_rolling_max_{window}h'] = df[col].rolling(window=window, min_periods=1).max()

print(f"✓ Created rolling window features for {len(sensor_cols)} sensors")
print(f"  Window sizes: {window_sizes} hours")


STATISTICAL FEATURE ENGINEERING
Creating rolling window features...
✓ Created rolling window features for 9 sensors
  Window sizes: [3, 6, 12] hours


In [13]:
print("\n" + "="*80)
print("LAG FEATURE ENGINEERING")
print("="*80)

# Create lag features for key sensors
lag_periods = [1, 3, 6, 12]  # hours

print("Creating lag features...")
for col in sensor_cols:
    for lag in lag_periods:
        df[f'{col}_lag_{lag}h'] = df[col].shift(lag)

# Fill initial NaN values with forward fill
df.fillna(method='bfill', inplace=True)

print(f"✓ Created lag features for {len(sensor_cols)} sensors")
print(f"  Lag periods: {lag_periods} hours")


LAG FEATURE ENGINEERING
Creating lag features...
✓ Created lag features for 9 sensors
  Lag periods: [1, 3, 6, 12] hours


In [14]:
print("\n" + "="*80)
print("RATE OF CHANGE FEATURES")
print("="*80)

# Calculate rate of change (derivative approximation)
for col in sensor_cols:
    df[f'{col}_rate_of_change'] = df[col].diff()
    df[f'{col}_rate_of_change_pct'] = df[col].pct_change() * 100

print(f"✓ Created rate of change features for {len(sensor_cols)} sensors")


RATE OF CHANGE FEATURES
✓ Created rate of change features for 9 sensors


In [15]:
print("\n" + "="*80)
print("STATISTICAL ANOMALY INDICATORS")
print("="*80)

# Z-score based anomaly indicators
for col in sensor_cols:
    mean = df[col].mean()
    std = df[col].std()
    df[f'{col}_zscore'] = (df[col] - mean) / (std + epsilon)
    df[f'{col}_is_outlier_3std'] = (np.abs(df[f'{col}_zscore']) > 3).astype(int)

# IQR-based outlier detection
for col in sensor_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[f'{col}_is_outlier_iqr'] = ((df[col] < lower_bound) | (df[col] > upper_bound)).astype(int)

print("✓ Created statistical anomaly indicators")
print("  - Z-score based (3-sigma rule)")
print("  - IQR-based outlier detection")


STATISTICAL ANOMALY INDICATORS
✓ Created statistical anomaly indicators
  - Z-score based (3-sigma rule)
  - IQR-based outlier detection


In [16]:
print("\n" + "="*80)
print("DOMAIN-SPECIFIC ANOMALY FLAGS")
print("="*80)

# Physical constraint violations
df['steam_pressure_negative_flag'] = (df['steam_pressure'] < 0).astype(int)
df['abnormal_water_flow'] = ((df['water_flow_rate'] < 0) | (df['water_flow_rate'] > 5)).astype(int)
df['abnormal_temperature'] = ((df['water_temperature'] < 0) | (df['water_temperature'] > 100)).astype(int)
df['low_efficiency_flag'] = (df['thermal_efficiency_proxy'] < df['thermal_efficiency_proxy'].quantile(0.1)).astype(int)
df['high_thermal_stress'] = (df['thermal_stress'] > df['thermal_stress'].quantile(0.9)).astype(int)

# Unusual pressure conditions
df['extreme_pressure_drop'] = (np.abs(df['pressure_drop']) > df['pressure_drop'].std() * 3).astype(int)

# Turbine anomalies
df['turbine_overspeed'] = (df['turbine_rpm'] > df['turbine_rpm'].quantile(0.95)).astype(int)
df['turbine_low_power'] = ((df['turbine_rpm'] > 50) & (df['power_output'] < 1)).astype(int)

print("✓ Created domain-specific anomaly flags:")
print("  - Physical constraint violations")
print("  - Efficiency anomalies")
print("  - Pressure anomalies")
print("  - Turbine performance flags")


DOMAIN-SPECIFIC ANOMALY FLAGS
✓ Created domain-specific anomaly flags:
  - Physical constraint violations
  - Efficiency anomalies
  - Pressure anomalies
  - Turbine performance flags


In [17]:
print("\n" + "="*80)
print("FEATURE SCALING")
print("="*80)

# Create scaled versions for ML models
# Standard Scaler (for normally distributed features)
scaler_standard = StandardScaler()
# Robust Scaler (for features with outliers)
scaler_robust = RobustScaler()

# Columns to scale (excluding timestamps and categorical)
cols_to_scale = [col for col in df.columns if col not in 
                 ['timestamp', 'shift', 'is_weekend', 'hour', 'day', 'day_of_week', 
                  'month', 'year'] and '_flag' not in col and '_is_outlier' not in col]

# Create scaled dataframe
df_scaled = df.copy()
df_scaled[cols_to_scale] = scaler_standard.fit_transform(df[cols_to_scale])

# Create robust scaled dataframe
df_robust_scaled = df.copy()
df_robust_scaled[cols_to_scale] = scaler_robust.fit_transform(df[cols_to_scale])

print(f"✓ Created scaled versions of dataset")
print(f"  - Standard Scaled: {df_scaled.shape}")
print(f"  - Robust Scaled: {df_robust_scaled.shape}")


FEATURE SCALING
✓ Created scaled versions of dataset
  - Standard Scaled: (2889, 236)
  - Robust Scaled: (2889, 236)


In [19]:
print("\n" + "="*80)
print("DATA QUALITY REPORT")
print("="*80)

# Replace infinite values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Count issues
infinite_count = df.isin([np.inf, -np.inf]).sum().sum()
nan_count = df.isnull().sum().sum()

print(f"Infinite values: {infinite_count}")
print(f"NaN values: {nan_count}")

if nan_count > 0:
    print("\nHandling NaN values...")
    # Forward fill then backward fill
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    


DATA QUALITY REPORT
Infinite values: 0
NaN values: 0


In [20]:
print("\n" + "="*80)
print("SAVING PREPROCESSED DATA")
print("="*80)

# Save main preprocessed dataset
df.to_csv('thermal_plant_preprocessed.csv', index=False)
print("✓ Saved: thermal_plant_preprocessed.csv")

# Save scaled versions
df_scaled.to_csv('thermal_plant_scaled.csv', index=False)
print("✓ Saved: thermal_plant_scaled.csv")

df_robust_scaled.to_csv('thermal_plant_robust_scaled.csv', index=False)
print("✓ Saved: thermal_plant_robust_scaled.csv")

# Save Power BI optimized version (selected features)
powerbi_cols = ['timestamp', 'hour', 'day_of_week', 'month', 'shift', 'is_weekend',
                'water_flow_rate', 'water_pressure', 'water_temperature',
                'steam_pressure', 'steam_temperature', 'fuel_rate_proxy',
                'turbine_rpm', 'power_output', 'generator_temperature_proxy',
                'thermal_efficiency_proxy', 'steam_quality', 'turbine_load_factor',
                'pressure_ratio', 'generator_load', 'fuel_efficiency',
                'water_flow_rate_rolling_mean_6h', 'power_output_rolling_std_6h',
                'turbine_rpm_rate_of_change', 'power_output_zscore',
                'steam_pressure_negative_flag', 'low_efficiency_flag', 
                'turbine_overspeed', 'turbine_low_power']

df_powerbi = df[powerbi_cols].copy()
df_powerbi.to_csv('thermal_plant_powerbi.csv', index=False)
print("✓ Saved: thermal_plant_powerbi.csv (Power BI optimized)")


SAVING PREPROCESSED DATA
✓ Saved: thermal_plant_preprocessed.csv
✓ Saved: thermal_plant_scaled.csv
✓ Saved: thermal_plant_robust_scaled.csv
✓ Saved: thermal_plant_powerbi.csv (Power BI optimized)
